In [281]:
# borrowed from Total3D repo -> notebooks/parse_OR_labels-lighting-reindexed%2Bassigned-V4.ipynb
import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/semanticInverse/train')

import os
import json

%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import matplotlib.pyplot as plt
%matplotlib widget

# dataset = 'sunrgbd'
dataset = 'OR'

mode = 'val'
# mode = 'train'
# mode = 'train-tmp'

# split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': 'utils_OR/openrooms/preprocessed-full'}
split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': '/home/ruizhu/Documents/Projects/semanticInverse/train/data/openrooms/list_OR_V4full/list'}

if dataset == 'sunrgbd':
    split_file = os.path.join(split_path_dict[dataset], mode + '.json')
    with open(split_file) as file:
        split = json.load(file)
elif dataset == 'OR':
    split_file = os.path.join(split_path_dict[dataset], mode + '.txt')
    with open(split_file) as f:
        mylist = f.read().splitlines() 
    split = [x.split(' ') for x in mylist]

print(len(split), split[0])

5664 ['scene0509_00', '1', 'mainDiffLight_xml1/scene0509_00/im_1.hdr', 'main_xml1/scene0509_00/imsemLabel_1.npy']


In [283]:
import torch
import numpy as np
from pathlib import Path
from utils.utils_total3D.utils_OR_visualize import format_layout, Box
from utils.utils_total3D.data_config import OR4XCLASSES_dict, NYU40CLASSES
from utils.utils_total3D.utils_others import OR4X_mapping_catInt_to_RGB, OR4X_mapping_catStr_to_RGB
from utils.utils_total3D.utils_OR_vis_labels import read_vis_scene_3d, RGB_to_01
import pickle

OR = 'OR45'
classes = OR4XCLASSES_dict[OR] if dataset == 'OR' else NYU40CLASSES
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catInt_to_RGB['light']), 'rb') as f:
    OR_mapping_catInt_to_RGB = pickle.load(f)[OR]
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catStr_to_RGB['light']), 'rb') as f:
    OR_mapping_catStr_to_RGB = pickle.load(f)[OR]

import random
# image_id = random.randint(0, len(split))
# image_id = 3
# pickle_path_ori = split[image_id]

# pickle_id = random.randint(0, len(split))
# pickle_id = 10
# pickle_id = 26
# pickle_id = 27

pickle_id = 16 # blue room, sofa, window
wall_idx, cell_i, cell_j = 2, 3, 3

# pickle_id = 6
# wall_idx, cell_i, cell_j = 4, 2, 4

# pickle_id = 18 # classroom
# wall_idx, cell_i, cell_j = 4, 3, 3

# random
pickle_id = 25


naming = split[pickle_id][2]

# naming = 'main_xml1/scene0552_00/im_1.hdr'
# naming = 'main_xml1/scene0552_00/im_1.hdr'
# naming = 'mainDiffLight_xml1/scene0017_02/im_4.hdr'
# naming = 'mainDiffMat_xml1/scene0695_03/im_26.hdr'
# pickle_path_ori = '/data/ruizhu/OR-V3-OR45_total3D_train_test_data/%06d.pkl'%pickle_id
meta_split = naming.split('/')[0]
scene_name = naming.split('/')[1]
withinsequence_id = int(naming.split('/')[2].split('.')[0].split('_')[1])
# assert pickle_id==withinsequence_id

train_test_data_path = '/data/ruizhu/OR-V4full-detachEmitterRERERE20210502-OR45_total3D_train_test_data'

pickle_path_ori = '%s/%s/%s/%s'%(train_test_data_path, naming.split('/')[0], naming.split('/')[1], naming.split('/')[2].replace('im', 'layout_obj').replace('hdr', 'pkl'))


grid_size = 8

pickle_path = pickle_path_ori.replace('.pkl', '_reindexed.pkl')
pickle_emitters_path = pickle_path_ori.replace('.pkl', '_emitters.pkl')
pickle_emitters_assign_info_list_path = pickle_path_ori.replace('.pkl', '_emitters_assign_info_%dX%d_V4.pkl'%(grid_size, grid_size))

print(pickle_path)

# ======= read saved results from evaluation
# results_folder = '20210427-142731--EVAL-20210427-011227'; tid=34000
# results_folder = '20210427-041449-train_POD_emitterEst_lightAccuV3_bs2x4_GTBRDF-light-Light_FIXFIXFIXED2axis_overfitValMini_RE2'; tid=12000

# EST
# results_folder = '20210427-025334-train_POD_emitterEst_lightAccuV3_bs2x4_RuitrainedBRDF-light-Light_FIXFIXFIXED2axis'; tid=44000
# GT
# results_folder = '20210427-025239-train_POD_emitterEst_lightAccuV3_bs4x4_GT-BRDF-Light_FIXFIXFIXED2axis'; tid=24000

# GT envmap input: 20210428-170647-train_POD_emitterEst_lightAccuV3_bs2x4_GTsampledEnvMap_FIXFIXFIXEDaxis
# results_folder = '20210429-214740--EVAL-20210428-170647'; tid=0

# test img sample
results_folder = '20210501-165504--tmp'; tid=0

layout_results_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_layout_info_tid%d-%d.pickle'%(results_folder, tid, pickle_id)
emitter_results_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_cells_vis_info_list_tid%d-%d.pickle'%(results_folder, tid, pickle_id)

with open(layout_results_pickle_path, 'rb') as f:
    layout_pickle_dict = pickle.load(f)
    print(layout_pickle_dict.keys())
with open(emitter_results_pickle_path, 'rb') as f:
    emitter_pickle_dict = pickle.load(f)
    print(emitter_pickle_dict.keys())

cells_vis_info_list = emitter_pickle_dict['cells_vis_info_list']
cam_R_pred = layout_pickle_dict['pre_cam_R']
pred_layout = layout_pickle_dict['pre_layout']
emitter_cls_prob_PRED = emitter_pickle_dict['emitter_cls_prob_PRED']
cell_info_grid_PRED = emitter_pickle_dict['cell_info_grid_PRED']

results_emitter_input_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_emitter_input_%d.pickle'%('20210501-170539--tmp', 0)
# results_emitter_input_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_emitter_input_%d.pickle'%(results_folder, pickle_id)

with open(results_emitter_input_pickle_path, 'rb') as f:
    results_emitter_input_pickle_dict = pickle.load(f)
    
env_scale, hdr_scale = results_emitter_input_pickle_dict['env_scale'], results_emitter_input_pickle_dict['hdr_scale']
print(env_scale, hdr_scale)

/data/ruizhu/OR-V4full-detachEmitterRERERE20210502-OR45_total3D_train_test_data/mainDiffLight_xml1/scene0608_01/layout_obj_22_reindexed.pkl


FileNotFoundError: [Errno 2] No such file or directory: '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/20210501-165504--tmp/results_layout_info_tid0-25.pickle'

In [284]:
import pickle
with open(pickle_path, 'rb') as f:
    sequence = pickle.load(f)
print(sequence.keys())

with open(pickle_emitters_path, 'rb') as f:
    sequence_emitters = pickle.load(f)
print(sequence_emitters.keys(), sequence_emitters['sequence_name'], sequence_emitters['withinsequence_id'])
with open(pickle_emitters_assign_info_list_path, 'rb') as f:
    sequence_emitter2wall_assign_info_dict = pickle.load(f)
print(sequence_emitter2wall_assign_info_dict.keys())

import pickle5 as pickle
emitter_representation_type = '0ambient'
emitters_prop_dict_representation_dict_path = pickle_path.replace('layout_obj', ('emitters_prop_dict_%s'%emitter_representation_type)).replace('_reindexed', '')
with open(emitters_prop_dict_representation_dict_path, 'rb') as f:
    emitters_prop_dict_representation_dict = pickle.load(f)

# transform_to_total3d_coords_dict = {'transform_R': frame.transform_R, 'transform_t': frame.transform_t}
transform_to_total3d_coords_dict_path = Path(train_test_data_path) / meta_split / scene_name / ('transform_to_total3d_coords_dict_%d.pkl'%withinsequence_id)
import pickle5 as pickle
with open(transform_to_total3d_coords_dict_path, 'rb') as f:
    transform_to_total3d_coords_dict = pickle.load(f)
    
# RAW_data_path = '/newfoundland2/ruizhu/siggraphasia20dataset/layout_labels_V4full-detachEmitterRERERE20210420/'
# emitter_dict_pickle_file = Path(RAW_data_path) / meta_split / scene_name / ('emitter_dict_%s_%d.pickle'%('0ambient', withinsequence_id))
# with open(emitter_dict_pickle_file, "rb") as f:
#     emitter_dict_RAW = pickle.load(f)


%matplotlib widget

gt_boxes_list = [format_layout(x) for x in sequence['boxes']['bdb3D']]
for x, class_id in zip(gt_boxes_list, sequence['boxes']['size_cls']):
    x.update({'class_id': class_id})
gt_boxes_dict = {}
for key in list(gt_boxes_list[0].keys()):
    gt_boxes_dict[key] = [x[key] for x in gt_boxes_list]
    
gt_layout = sequence['layout']['bdb3D']

from utils.utils_total3D.data_config import Dataset_Config
from utils.utils_total3D.utils_OR_layout import to_dict_tensor

from utils.utils_total3D.utils_OR_cam import get_rotation_matrix_gt

OR = 'OR45'
dataset_config = Dataset_Config(dataset, OR=OR, version='V3', paths={'total3D_lists_path': '../train/data/openrooms/list_OR_V4full', 'layout_emitter_path': '/data/ruizhu/OR-V4full-OR45_total3D_train_test_data'})
bins_tensor = to_dict_tensor(dataset_config.bins, if_cuda=False)
cam_R_gt = get_rotation_matrix_gt(bins_tensor,
                                  torch.zeros([1]).long()+sequence['camera']['pitch_cls'], torch.zeros([1])+sequence['camera']['pitch_reg'],
                                  torch.zeros([1]).long()+sequence['camera']['roll_cls'], torch.zeros([1])+sequence['camera']['roll_reg']).squeeze().numpy()

# ======== emitters_list
emitters_obj_list = []
for x in range(sequence_emitters['boxes']['bdb3D'].shape[0]):
    obj_dict_new = {'obj_box_3d': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'random_id': sequence_emitters['boxes']['random_id'][x], \
                    'emitter_prop': sequence_emitters['boxes']['emitter_prop'][x], \
                    'bdb3D_emitter_part': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'cat_id': sequence_emitters['boxes']['size_cls'][x], \
                    'cat_name': classes[sequence_emitters['boxes']['size_cls'][x]], 'cat_color': RGB_to_01(OR_mapping_catInt_to_RGB[sequence_emitters['boxes']['size_cls'][x]])}
    obj_random_id = sequence_emitters['boxes']['random_id'][x]
    emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']
    if sequence_emitters['boxes']['emitter_prop'][x]['obj_type'] == 'window':
        light_center_world_total3d = emitter_prop_total3d['light_center_world_total3d'].reshape(3, 1)
        light_axis_world_total3d = emitter_prop_total3d['light_axis_world_total3d'].reshape(3, 1)
        print(light_axis_world_total3d)
    else:
        light_center_world_total3d = np.zeros((3, 1), dtype=np.float32)
        light_axis_world_total3d = np.zeros((3, 1), dtype=np.float32)
    obj_dict_new['light_world_total3d_centeraxis'] = [light_center_world_total3d, light_axis_world_total3d]
    
    obj_dict_new['emitter_prop'].update({'emitter_rgb_float': emitter_prop_total3d['intensity']})

    emitters_obj_list.append(obj_dict_new)
    
emitter2wall_assign_info_list = sequence_emitter2wall_assign_info_dict['emitter2wall_assign_info_list']
cell_info_grid_GT_includeempty = sequence_emitter2wall_assign_info_dict['cell_info_grid']
cell_info_grid_GT = []
for wall_idxx in range(6):
    for i in range(grid_size):
        for j in range(grid_size):
            cell_info = cell_info_grid_GT_includeempty[wall_idxx * grid_size**2 + i * grid_size + j]
            if cell_info['obj_type'] is None:
                continue
            cell_info['wallidx_i_j'] = (wall_idxx, i, j)
            cell_info_grid_GT.append(cell_info)
                
            obj_random_id = cell_info['emitter_info']['random_id']
            emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']


#             cell_intensity = np.array([emitter_prop_total3d['intensity_scale255'] * x * 255.for x in emitter_prop_total3d['intensity_scaled01']]) # intensity_scaled: [0., 1.]
            cell_intensity = np.array(emitter_prop_total3d['intensity']) * env_scale * hdr_scale
            cell_info['emitter_info']['intensity_scalelog'] = np.log(np.clip(np.linalg.norm(cell_intensity.flatten()) + 1., 1., np.inf))
            cell_info['emitter_info']['intensity'] = [x * env_scale * hdr_scale for x in emitter_prop_total3d['intensity']]
            intensity_scale255 = max(cell_info['emitter_info']['intensity']) / 255.
            intensity_scaled01 = [np.clip(x / (intensity_scale255+1e-5) / 255., 0., 1.) for x in cell_info['emitter_info']['intensity']]
            cell_info['emitter_info']['intensity_scaled01'] = intensity_scaled01

            # other representation-specific params
            if cell_info['obj_type'] == 'window':
                cell_info['emitter_info']['lamb'] = emitter_prop_total3d['lamb']
                cell_info['emitter_info']['light_dir_abs'] = emitter_prop_total3d['light_axis_world_total3d'].reshape(3,)
                if emitter_representation_type in ['1ambient']:
                    cell_info['emitter_info']['ambient'] = emitter_prop_total3d['ambient']
                if emitter_representation_type in ['2ambient']:
                    cell_info['emitter_info']['ambientL'] = emitter_prop_total3d['ambientL']
                    cell_info['emitter_info']['ambientR'] = emitter_prop_total3d['ambientR']



plt.figure(figsize=(15, 8))
ax_2d = plt.gca()

grid_prob = sequence_emitter2wall_assign_info_dict['wall_grid_prob'].reshape(6, -1)
import imageio
rgb_img = imageio.imread(sequence['rgb_img_path'])

scene_box = Box(rgb_img, None, sequence['camera']['K'], cam_R_gt, cam_R_pred, gt_layout, gt_layout, gt_boxes_dict, None, 'GT', None, dataset=dataset, OR='OR45', \
                emitters_obj_list=emitters_obj_list, emitter2wall_assign_info_list=emitter2wall_assign_info_list, \
                emitter_cls_prob_PRED=emitter_cls_prob_PRED, emitter_cls_prob_GT=grid_prob, cell_info_grid_GT=cell_info_grid_GT, cell_info_grid_PRED=cell_info_grid_PRED, \
                grid_size=grid_size, \
                paths={'color_file': '../train/data/openrooms/total3D_colors/OR4X_mapping_catInt_to_RGB_light.pkl'})
if_use_plt=True
img_map, _ = scene_box.draw_projected_layout('GT', return_plt=True, if_save=False, save_path='', if_use_plt=if_use_plt, fig_or_ax=ax_2d)
img_map2 = scene_box.draw_projected_bdb3d('GT', return_plt=True, if_save=False, save_path='', if_use_plt=if_use_plt, fig_or_ax=ax_2d)
# plt.show()

fig_3d, ax_3ds_scene = scene_box.draw_3D_scene_plt('both', if_show_objs=False, hide_random_id=False, if_print_log=False)
# ax_3d = ax_3ds[0]
# ax_3ds_scene[0].view_init(elev=-46, azim=86)

# %matplotlib widget
# sys.path.insert(0, '/home/ruizhu/Documents/Projects/semanticInverse/train/SimpleLayout')
# depth_combined, mask_conflict = scene_box.draw_projected_depth('GT', return_plt=True, if_save=False, save_path='', if_vis=True)

wall_cell_idxes = []
for x in cell_info_grid_GT:
    if x['obj_type'] == 'window':
        wall_cell_idxes.append(x['wallidx_i_j'])
print(wall_cell_idxes)

dict_keys(['rgb_img_path', 'depth_map', 'boxes', 'camera', 'layout', 'scene_name', 'withinsequence_id', 'meta_split', 'meta_name', 'sub_name', 'scene_pickle_file', 'cam_pickle_file', 'frame_pickle_file', 'reindex_info_dict'])
dict_keys(['sequence_name', 'withinsequence_id', 'boxes', 'emitter_random_ids']) scene0608_01 22
dict_keys(['emitter2wall_assign_info_list', 'emitters_obj_list', 'wall_grid_prob', 'cell_prob_mean', 'cell_prob', 'cell_count', 'cell_info_grid'])
[[-0.8241]
 [ 0.0553]
 [ 0.5637]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[draw_projected_layout] Returned.
 Detected for 32:lamp not in valid_class_ids of dataset: OR
[draw_projected_bdb3d] Returned.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(5, 2, 3), (5, 2, 4), (5, 2, 5), (5, 2, 6), (5, 2, 7), (5, 3, 3), (5, 3, 4), (5, 3, 5), (5, 3, 6), (5, 3, 7), (5, 4, 3), (5, 4, 4), (5, 4, 5), (5, 4, 6), (5, 4, 7), (5, 5, 3), (5, 5, 4), (5, 5, 5), (5, 5, 6), (5, 5, 7), (5, 6, 3), (5, 6, 4), (5, 6, 5), (5, 6, 6), (5, 6, 7)]


# Diff project emitters

In [285]:
from SimpleLayout.utils_SL_torch import SimpleSceneTorch

cam_K = sequence['camera']['K'].astype(np.float32)
im_width = 320
im_height = 240
cam_K_ratio = cam_K[0][2] / (im_width/2.)
cam_K = cam_K / cam_K_ratio

bdb_sample = torch.from_numpy(gt_layout).cuda().float()

cam_dict = {'origin': torch.tensor([0., 0., 0.]).cuda(), 'cam_axes': torch.from_numpy(cam_R_gt.T).cuda(), \
            'f_x': torch.tensor(cam_K[0][0]).cuda(), 'f_y': torch.tensor(cam_K[1][1]).cuda(), 'width': int(im_width), 'height': int(im_height)}
simpleSceneTorch = SimpleSceneTorch(cam_dict, bdb_sample)
edges_front_list, face_edges_list, face_verts_list = simpleSceneTorch.get_edges_front(ax_3d=None, if_vis=False)
mask_combined, mask_list, mask_conflict = simpleSceneTorch.poly_to_masks(face_verts_list)

invd_list = simpleSceneTorch.param_planes()

%matplotlib widget
ax_2d = simpleSceneTorch.vis_2d_bbox_proj(bdb_sample.detach().cpu().numpy(), edge_list=[x[0] for x in edges_front_list], if_show=True)
simpleSceneTorch.vis_mask_combined(mask_combined.cpu().numpy(), ax_2d=ax_2d)

# fig = plt.figure(figsize=(15, 4))
# plt.subplot(121)
# plt.imshow(depth_combined.detach().cpu().numpy(), cmap='jet')
# plt.colorbar()
# plt.subplot(122)
# plt.imshow(mask_conflict, cmap='jet')
# plt.colorbar()
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

## batch mode + cells

In [286]:
from SimpleLayout.SimpleSceneTorchBatch import SimpleSceneTorchBatch
import cv2

cam_K = sequence['camera']['K'].astype(np.float32)
im_width = 320
im_height = 240
# im_width = 640
# im_height = 480
cam_K_ratio = cam_K[0][2] / (im_width/2.)
cam_K = np.vstack([cam_K[:2, :] / cam_K_ratio, cam_K[2:3, :]])
print(cam_K)

gt_layout_torch = torch.from_numpy(gt_layout).cuda().float()
gt_layout_torch = gt_layout_torch.unsqueeze(0).repeat(2, 1, 1)


cam_dict = {'origin': torch.tensor([0., 0., 0.]).cuda(), 'cam_K': torch.from_numpy(cam_K).cuda()}
cam_axes_batch = torch.from_numpy(cam_R_gt).cuda().unsqueeze(0).repeat(2, 1, 1)
simpleSceneTorch = SimpleSceneTorchBatch(cam_dict, im_height=im_height, im_width=im_width)

simpleSceneTorch.form_camera(cam_axes_batch)
verts_proj, front_flags = simpleSceneTorch.transform_and_proj(gt_layout_torch)

verts_proj_vis = verts_proj[0].cpu().numpy()
rgb_img_resize = cv2.resize(rgb_img, (im_width, im_height), interpolation = cv2.INTER_AREA )

%matplotlib widget

fig = plt.figure(figsize=(15, 4))
# plt.subplot(121)
plt.imshow(rgb_img_resize)
for vert in verts_proj_vis:
    plt.scatter(vert[0], vert[1])
plt.xlim([0, im_width])
plt.ylim([im_height, 0])
plt.show()

[[288.9354   0.     160.    ]
 [  0.     288.9354 120.    ]
 [  0.       0.       1.    ]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [289]:
cells_verts_all = simpleSceneTorch.layout_to_cells(gt_layout_torch)
cells_verts_all_center = cells_verts_all.mean(axis=-1) # [B, 6, 8, 8, 3]
batch_size = cells_verts_all_center.shape[0]

# wall_idx_vis = wall_idx
wall_idx_vis = 2
verts_proj, front_flags = simpleSceneTorch.transform_and_proj(cells_verts_all_center[:, wall_idx_vis].reshape(batch_size, -1, 3)) # assuming align_corners=False, meaning the upper-left corner is 0 and lower corner is (W, H)
# verts_proj, front_flags = simpleSceneTorch.transform_and_proj(cells_verts_all_center.reshape(batch_size, -1, 3)) # assuming align_corners=False, meaning the upper-left corner is 0 and lower corner is (W, H)

# verts_proj = verts_proj * front_flags.unsqueeze(-1).float())

verts_proj_vis = verts_proj[0].cpu().numpy()
verts_proj_vis = verts_proj_vis[front_flags[0].cpu().numpy()]
rgb_img_resize = cv2.resize(rgb_img, (im_width, im_height), interpolation = cv2.INTER_AREA )

%matplotlib widget

fig = plt.figure(figsize=(6, 4))
# plt.subplot(121)
plt.imshow(rgb_img_resize)
for vert_idx, vert in enumerate(verts_proj_vis):
#     color= 'g' if front_flags[0].cpu().numpy()[vert_idx] else 'r'
    color = 'g'
    plt.scatter(vert[0], vert[1], color=color)
plt.xlim([0, im_width])
plt.ylim([im_height, 0])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [275]:
feat_height, feat_width = im_height, im_width
feat_map = torch.from_numpy(rgb_img_resize).permute(2, 0, 1).unsqueeze(0).repeat(2, 1, 1, 1).cuda().float()/255. # use image intensity as feat map for debugging purposes
feat_map = feat_map.unsqueeze(2) # [B, 3, 1, H, W]

verts_proj[torch.logical_not(front_flags.unsqueeze(-1).repeat(1, 1, 2))] = -100

verts_proj_reshape = verts_proj.view(batch_size, 6, grid_size, grid_size, 2) # [B, 6, 8, 8, 2]
wh_tensor = torch.tensor([feat_width, feat_height]).reshape(1, 1, 1, 1, 2).cuda().float()
verts_proj_reshape_normalized = verts_proj_reshape/wh_tensor * 2. - 1.
verts_proj_reshape_normalized_concat = torch.cat([verts_proj_reshape_normalized, torch.zeros(batch_size, 6, grid_size, grid_size, 1).cuda()], -1) # [B, 6, 8, 8, 3]

# [B, D, 1, H, W], [B, 6, 8, 8, 3] -> [B, D, 6, 8, 8]
feat_map_resampled = torch.nn.functional.grid_sample(feat_map, verts_proj_reshape_normalized_concat, padding_mode="zeros", align_corners=False)

feat_map_resampled.shape



torch.Size([2, 3, 6, 8, 8])

In [279]:
grid_size = 8

color_array_full = feat_map_resampled[0].permute(1, 2, 3, 0).cpu().numpy()

# Option 2: read from saved array
# feat_map_resampled_saved = results_emitter_input_pickle_dict['feat_map_resampled']
# color_array_full = feat_map_resampled_saved[-3:].transpose(1, 2, 3, 0)
# color_array_full = color_array_full**(1./2.2)

lightnet_array_GT = color_array_full.reshape(6, grid_size, grid_size, -1)

%matplotlib widget

scene_box = Box(rgb_img, None, sequence['camera']['K'], cam_R_gt, None, gt_layout, None, gt_boxes_dict, None, 'GT', None, dataset=dataset, OR='OR45', \
                emitters_obj_list=emitters_obj_list, emitter2wall_assign_info_list=emitter2wall_assign_info_list, \
                emitter_cls_prob_PRED=np.random.random((6, grid_size**2)), emitter_cls_prob_GT=grid_prob, cell_info_grid_GT=cell_info_grid_GT, grid_size=grid_size, \
                paths={'color_file': '../train/data/openrooms/total3D_colors/OR4X_mapping_catInt_to_RGB_light.pkl'})

fig_3d, ax_3d_scene = scene_box.draw_3D_scene_plt('GT', hide_cells=False, if_show_objs=False, scale_emitter_length=2.)
ax_3d_scene[1] = fig_3d.add_subplot(122, projection='3d')
scene_box.draw_3D_scene_plt('GT', fig_or_ax=[ax_3d_scene[1], ax_3d_scene[0]], hide_cells=False, if_show_objs=False, scale_emitter_length=2., if_print_log=True)

scene_box.draw_all_cells(ax_3d_scene[1], gt_layout, lightnet_array_GT=lightnet_array_GT, alpha=0.9, highlight_cells=[])

az = -16
elev = 28
ax_3d_scene[1].view_init(elev=elev, azim=az)
ax_3d_scene[0].view_init(elev=elev, azim=az)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[[[0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         ...,
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ]],

        [[0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         ...,
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ]],

        [[0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         ...,
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ]],

        ...,

        [[0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         ...,
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ]],

        [[0.    , 0.    , 0.    ],
         [0.    , 0.    , 0.    ],
         [0.